# Assistant-01<br/>The OpenAI reference demo

Reference:

- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)
- [Github-OpenAI Python/examples/Assistant demo notebook](https://github.com/openai/openai-python/blob/main/examples/assistant.py)


### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import time
import oaihelper as helper

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Create an assistant

In [2]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=helper.gpt_deployment_name
)
assistant

Assistant(id='asst_nZP1Me7pypD9Q1t5orLMTDqb', created_at=1705510118, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt4-1106-fta-assistants', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

### Create an assistant thread

In [3]:
thread = client.beta.threads.create()
thread

Thread(id='thread_P3rT3ZlKo5cqbwlaXWtkICx0', created_at=1705510121, metadata={}, object='thread')

### Create a user message in the assistant thread

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"    
)
message

ThreadMessage(id='msg_XkhhqMGwcm2pSrnN4vn1Akn8', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1705510130, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_P3rT3ZlKo5cqbwlaXWtkICx0')

## Run the assistant in the assistant thread

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
run

Run(id='run_nOpMdUzzQZDYvxll4OPAm3aJ', assistant_id='asst_nZP1Me7pypD9Q1t5orLMTDqb', cancelled_at=None, completed_at=None, created_at=1705510134, expires_at=1705510734, failed_at=None, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt4-1106-fta-assistants', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_P3rT3ZlKo5cqbwlaXWtkICx0', tools=[ToolAssistantToolsCode(type='code_interpreter')])

### Wait until the run is completed

In [6]:
print("checking assistant status. ")
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)

        print("messages: ")
        for message in messages:
            assert message.content[0].type == "text"
            print({"role": message.role, "message": message.content[0].text.value})

        break
    else:
        print("in progress...")
        time.sleep(5)

checking assistant status. 
messages: 
{'role': 'assistant', 'message': 'Of course, Jane Doe! To solve the equation `3x + 11 = 14`, we need to find the value of x that makes this equation true.\n\nThe steps to solve this equation are as follows:\n\n1. Subtract 11 from both sides of the equation to isolate the term with x on one side:\n\\[ 3x + 11 - 11 = 14 - 11 \\]\n\n2. Simplify both sides:\n\\[ 3x = 3 \\]\n\n3. Divide both sides by 3 to solve for x:\n\\[ \\frac{3x}{3} = \\frac{3}{3} \\]\n\n4. The solution is:\n\\[ x = 1 \\]\n\nSo, the value of x that satisfies the equation is 1. Would you like me to perform this calculation explicitly using Python?'}
{'role': 'user', 'message': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'}


### List all running assistants in the model

In [7]:
# List the registered assistants
assistants = client.beta.assistants.list()
count = 0
for assistant in assistants.data:
    print(assistant)
    count += 1
    if count ==2:
        break

# TODO: Do the assistants in the model and the threads, that have are not not disposed
# have a time limit or does there have to be a cleanup?


Assistant(id='asst_nZP1Me7pypD9Q1t5orLMTDqb', created_at=1705510118, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt4-1106-fta-assistants', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_TmNrLwSemy0F3kuf0NxPrnUz', created_at=1705433552, description=None, file_ids=[], instructions='You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.', metadata={}, model='gpt4-1106-fta-assistants', name='fta-Securities Trading Assistant', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='get_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['symbol']}), type='function'), ToolFunction(f

### Cleanup: clean the assistant and the thread thread

In [8]:
# TODO: Should the assistant live for the life of the application?
print(client.beta.assistants.delete(assistant.id))

# TODO: Should the thread live for the life of the execution?
print(client.beta.threads.delete(thread.id))

AssistantDeleted(id='asst_TmNrLwSemy0F3kuf0NxPrnUz', deleted=True, object='assistant.deleted')
ThreadDeleted(id='thread_P3rT3ZlKo5cqbwlaXWtkICx0', deleted=True, object='thread.deleted')
